In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from matplotalt import *

In [2]:
vistext_id_to_mpl_code = np.load("./evaluation/vistext_to_mpl/vistext_id_to_mpl_code.npy", allow_pickle=True).item()
vistext_val_data = pd.read_json("./evaluation/vistext_to_mpl/vistext_data_validation.json")
vistext_id_to_captions = defaultdict(list)
vistext_id_to_chart_type = {}
def get_id_to_captions(row):
    vistext_id_to_captions[row["img_id"]].append({"L1": row["caption_L1"], "L2L3": row["caption_L2L3"]})
    vistext_id_to_chart_type[row["img_id"]] = row["L1_properties"][0]

vistext_val_data.apply(get_id_to_captions, axis=1);
np.save("./evaluation/vistext_to_mpl/vistext_id_to_captions", vistext_id_to_captions)

In [4]:
#print(len(vistext_id_to_captions))
n_passed = 0
n_errors = 0
#vistext_id_to_captions = np.load("./evaluation/vistext_to_mpl/vistext_id_to_captions.npy", allow_pickle=True).item()
vistext_id_to_matplotalt_captions = {}
for chart_id in tqdm(vistext_id_to_captions.keys()):
    #print(vistext_id_to_mpl_code.keys())
    try:
        mpl_code = vistext_id_to_mpl_code[str(chart_id)]
        chart_type = vistext_id_to_chart_type[chart_id]
        exec(mpl_code)
        #print(mpl_code)
        matplotalt_caption = show_with_alt(desc_level=3, chart_type=chart_type, methods=[], stats=["max"], max_color_desc_count=0, return_alt=True)
        #print(matplotalt_caption)
        plt.clf()
        vistext_id_to_matplotalt_captions[chart_id] = matplotalt_caption
        n_passed += 1
    except Exception as e:
        #raise e
        #print(chart_id, mpl_code)
        n_errors += 1
        #print(e)

print(f"Num passed: {n_passed}")
print(f"Num errors: {n_errors}")
np.save("./evaluation/vistext_to_mpl/vistext_id_to_matplotalt_captions", vistext_id_to_matplotalt_captions)

  0%|          | 0/883 [00:00<?, ?it/s]

100%|██████████| 883/883 [00:32<00:00, 27.28it/s]

Num passed: 875
Num errors: 8


<Figure size 640x480 with 0 Axes>

In [ ]:
vistext_id_to_matplotalt_captions = np.load("./evaluation/vistext_to_mpl/vistext_id_to_matplotalt_captions.npy", allow_pickle=True).item()
vistext_id_to_gpt4v_captions = {}
num_captions = 0
for chart_id in tqdm(vistext_id_to_matplotalt_captions.keys()):
    mpl_code = vistext_id_to_mpl_code[str(chart_id)]
    chart_type = vistext_id_to_chart_type[chart_id]
    exec(mpl_code)
    gpt4v_caption = show_with_api_alt(chart_type=chart_type, methods=[], return_alt=True)
    #print(gpt4v_caption)
    plt.clf()
    vistext_id_to_gpt4v_captions[chart_id] = gpt4v_caption
    np.save("./evaluation/vistext_to_mpl/vistext_id_to_gpt4v_captions", vistext_id_to_gpt4v_captions)
    num_captions += 1
    if num_captions >= 100:
        break